# Deploy Specific Model Version to Red Hat OpenShift Data Science

This notebook deploys the model.joblib file to OpenShift Data Science through Inference Service.
It is required that the model.joblib and inference-service.yaml file exists.
In the pipeline, the model.joblib file is generated by the preceding job.

In [1]:
import os, time, subprocess
from jinja2 import Template

In [2]:
template_data = {"model_version": os.environ["model_version"]}
template = Template(open("inference-service.yaml").read())
rendered_template = template.render(template_data)
print('Rendered Template: \r' + rendered_template)

apiVersion: serving.kserve.io/v1beta1
kind: InferenceService
metadata:
  name: wine-model
  namespace: wines
  labels:
    name: model-23.08.06-120013
    opendatahub.io/dashboard: 'true'
  annotations:
    openshift.io/display-name: model-23.08.06-120013
    serving.kserve.io/deploymentMode: ModelMesh
spec:
  predictor:
    model:
      modelFormat:
        name: sklearn
        version: '0'
      runtime: wine
      storage:
        key: aws-connection-wine-models
        path: model-23.08.06-120013


In [3]:
subprocess.run(['oc', 'whoami'])
ps = subprocess.Popen(['echo', rendered_template], stdout=subprocess.PIPE)
output = subprocess.check_output(['oc', 'apply', '-f', '-'], stdin=ps.stdout)
ps.wait()
print(output)
print('Model deployment completed. Version: ' + os.environ["model_version"])

system:serviceaccount:wines:wines
b'inferenceservice.serving.kserve.io/wine-model unchanged\n'
Model deployment completed. Version: 23.08.06-120013
